Working with Data

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# download training data from open datasets
training_data = datasets.FashionMNIST(
    root= "data",
    train= True,
    download= True,
    transform= ToTensor(),
)

# download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),

)

100%|██████████| 26421880/26421880 [00:05<00:00, 5272864.54it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 209929.65it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:06<00:00, 736597.45it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4147575.30it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [8]:
# passing the dataset created as an argument to DataLoader

batch_size = 64

# Create data laoders
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C , H ,W]: {X.shape}")
  print(f"Shape of y: {y.shape} Type of y: {y.dtype}")
  break


Shape of X [N, C , H ,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) Type of y: torch.int64


Creating Models

In [9]:
# To create a neural network in PyTorch, we create a aclass that inherits form nn.Module.
# We define the layers of the network in the __init__ function.
# We specify how data will pass through the network in the forward function.
#To acceletate operations in the nerual network, we move it to the GPU or MPS if available.

In [10]:
# get the device for training (cpu, grpu or mps)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"The used device is {device}")

The used device is cpu


In [46]:
# define the desired model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
        )

  def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

model = NeuralNetwork().to(device)
print(f"Model is: {model}")


Model is: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimizing the model parameters

In [26]:
# To train a model, we need a loss function and an optimizer
#The loss function/the cost function/objective function, measures how well the model's predictions match actual data
#The optimizing function is algorithm that adjusts the model's parameters (weights and biases) to minimize the loss function.

In [47]:
loss_fn =  nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-3)

In [48]:
# In a single training loop, the model makes predictions on the training dataset (fed to it in batches)
# Backpropagates tge prediction error to adjust the model's parameters.

In [49]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X , y = X.to(device), y.to(device)

    # Computing the prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Bakcpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f" Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [55]:
# Checking the models performance agaisnt the test dataset to ensure it is learning

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f" Test: Error \n Accuracy: {(100*correct):>0.1f}%, Average loss: {test_loss:>8f} \n")

In [56]:
# The training process is done over several epochs(iterations)
# Each epoch makes the model learn parameters to make better predictions.
# We pring model accuracy and loss at each epoch
# We'd like to see teh accuracy increase and the loss decreasy after every epoch

In [57]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
---------------------------
 Loss: 1.924066 [   64/60000]
 Loss: 1.879750 [ 6464/60000]
 Loss: 1.769688 [12864/60000]
 Loss: 1.796815 [19264/60000]
 Loss: 1.704427 [25664/60000]
 Loss: 1.650053 [32064/60000]
 Loss: 1.666553 [38464/60000]
 Loss: 1.578768 [44864/60000]
 Loss: 1.613806 [51264/60000]
 Loss: 1.487571 [57664/60000]
 Test: Error 
 Accuracy: 60.5%, Average loss: 1.514545 

Epoch 2
---------------------------
 Loss: 1.590173 [   64/60000]
 Loss: 1.543150 [ 6464/60000]
 Loss: 1.402555 [12864/60000]
 Loss: 1.463452 [19264/60000]
 Loss: 1.358606 [25664/60000]
 Loss: 1.344405 [32064/60000]
 Loss: 1.357812 [38464/60000]
 Loss: 1.293077 [44864/60000]
 Loss: 1.334287 [51264/60000]
 Loss: 1.220239 [57664/60000]
 Test: Error 
 Accuracy: 63.7%, Average loss: 1.251484 

Epoch 3
---------------------------
 Loss: 1.334097 [   64/60000]
 Loss: 1.306319 [ 6464/60000]
 Loss: 1.147483 [12864/60000]
 Loss: 1.245674 [19264/60000]
 Loss: 1.128456 [25664/60000]
 Loss: 1.145489 [32064/60000

Saving Models

In [ ]:
# saving model serializing the internal state dictionary (containing the model parameters)

In [58]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorchModel Sarare to model.pth")

Saved PyTorchModel Sarare to model.pth


Loading Models

In [59]:
# loading process includes re-creating the model structure and loading the state dictionary into it

In [61]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

Now the model is ready to be used to make predictions

In [67]:
classes = [
    "T=shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
    ]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x =x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
